## <font color=#6B49F5> A Simple Implementation of FedAvg with PyTorch on IID Data </font> 
Please see https://towardsdatascience.com/federated-learning-a-simple-implementation-of-fedavg-federated-averaging-with-pytorch-90187c9c9577 for more details.

In [2]:
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier
import random
import math
from torch.utils.tensorboard import SummaryWriter
from matplotlib import pyplot

from pathlib import Path
import requests
import pickle
import gzip

import torch
import math
import torch.nn.functional as F
from torch import nn
from torch import optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from imblearn.over_sampling import SMOTE
import copy
from sklearn.metrics import confusion_matrix

pd.options.display.float_format = "{:,.4f}".format
sm = SMOTE(random_state=42)

In [3]:
THREAT_TYPE = 'threat_type'
THREAT_HL = 'threat_hl'

learning_rate = 0.01
numEpoch = 20
batch_size = 32
momentum = 0.9
print_amount=3
number_of_slices = 2
isSmote = False
runtime = 21

file_name = "federated_" + str(isSmote) + "_" + str(number_of_slices)  + "_" + str(runtime) + ".txt"
file = open(file_name, "w")

data_path = "D:\\learning\\PyTorch\\NSL_KDD-master\\"

colnames = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land',
            'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
            'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files',
            'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate',
            'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate',
            'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate',
            'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
            'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
            'dst_host_srv_rerror_rate', 'threat_type']

In [4]:
df_train = pd.read_csv(data_path + "KDDTrain+.csv", header = None)
df_train = df_train.iloc[:, :-1]

df_test = pd.read_csv(data_path + "KDDTest+.csv", header = None)
df_test = df_test.iloc[:, :-1]

df_train.columns = colnames
df_test.columns = colnames

df_train.loc[(df_train['threat_type'] == 'back'), 'threat_type'] = 1
df_train.loc[(df_train['threat_type'] == 'buffer_overflow'), 'threat_type'] = 2
df_train.loc[(df_train['threat_type'] == 'ftp_write'), 'threat_type'] = 3
df_train.loc[(df_train['threat_type'] == 'guess_passwd'), 'threat_type'] = 3
df_train.loc[(df_train['threat_type'] == 'imap'), 'threat_type'] = 3
df_train.loc[(df_train['threat_type'] == 'ipsweep'), 'threat_type'] = 4
df_train.loc[(df_train['threat_type'] == 'land'), 'threat_type'] = 1
df_train.loc[(df_train['threat_type'] == 'loadmodule'), 'threat_type'] = 2
df_train.loc[(df_train['threat_type'] == 'multihop'), 'threat_type'] = 3
df_train.loc[(df_train['threat_type'] == 'neptune'), 'threat_type'] = 1
df_train.loc[(df_train['threat_type'] == 'nmap'), 'threat_type'] = 4
df_train.loc[(df_train['threat_type'] == 'perl'), 'threat_type'] = 2
df_train.loc[(df_train['threat_type'] == 'phf'), 'threat_type'] = 3
df_train.loc[(df_train['threat_type'] == 'pod'), 'threat_type'] = 1
df_train.loc[(df_train['threat_type'] == 'portsweep'), 'threat_type'] = 4
df_train.loc[(df_train['threat_type'] == 'rootkit'), 'threat_type'] = 2
df_train.loc[(df_train['threat_type'] == 'satan'), 'threat_type'] = 4
df_train.loc[(df_train['threat_type'] == 'smurf'), 'threat_type'] = 1
df_train.loc[(df_train['threat_type'] == 'spy'), 'threat_type'] = 3
df_train.loc[(df_train['threat_type'] == 'teardrop'), 'threat_type'] = 1
df_train.loc[(df_train['threat_type'] == 'warezclient'), 'threat_type'] = 3
df_train.loc[(df_train['threat_type'] == 'warezmaster'), 'threat_type'] = 3
df_train.loc[(df_train['threat_type'] == 'normal'), 'threat_type'] = 0
df_train.loc[(df_train['threat_type'] == 'unknown'), 'threat_type'] = 6

df_test.loc[(df_test['threat_type'] == 'back'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'buffer_overflow'), 'threat_type'] = 2
df_test.loc[(df_test['threat_type'] == 'ftp_write'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'guess_passwd'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'imap'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'ipsweep'), 'threat_type'] = 4
df_test.loc[(df_test['threat_type'] == 'land'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'loadmodule'), 'threat_type'] = 2
df_test.loc[(df_test['threat_type'] == 'multihop'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'neptune'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'nmap'), 'threat_type'] = 4
df_test.loc[(df_test['threat_type'] == 'perl'), 'threat_type'] = 2
df_test.loc[(df_test['threat_type'] == 'phf'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'pod'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'portsweep'), 'threat_type'] = 4
df_test.loc[(df_test['threat_type'] == 'rootkit'), 'threat_type'] = 2
df_test.loc[(df_test['threat_type'] == 'satan'), 'threat_type'] = 4
df_test.loc[(df_test['threat_type'] == 'smurf'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'spy'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'teardrop'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'warezclient'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'warezmaster'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'normal'), 'threat_type'] = 0
df_test.loc[(df_test['threat_type'] == 'unknown'), 'threat_type'] = 6
df_test.loc[(df_test['threat_type'] == 'mscan'), 'threat_type'] = 4
df_test.loc[(df_test['threat_type'] == 'apache2'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'snmpgetattack'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'processtable'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'httptunnel'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'ps'), 'threat_type'] = 2
df_test.loc[(df_test['threat_type'] == 'snmpguess'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'mailbomb'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'named'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'sendmail'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'xterm'), 'threat_type'] = 2
df_test.loc[(df_test['threat_type'] == 'xlock'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'xsnoop'), 'threat_type'] = 3
df_test.loc[(df_test['threat_type'] == 'sqlattack'), 'threat_type'] = 2
df_test.loc[(df_test['threat_type'] == 'udpstorm'), 'threat_type'] = 1
df_test.loc[(df_test['threat_type'] == 'saint'), 'threat_type'] = 4
df_test.loc[(df_test['threat_type'] == 'worm'), 'threat_type'] = 1

df_full = pd.concat([df_train, df_test])

print('Attack types in full set: \n', df_full[THREAT_TYPE].value_counts())

Attack types in full set: 
 0    77053
1    53387
4    14077
3     3880
2      119
Name: threat_type, dtype: int64


In [5]:

print('Before normalization shape of data set : ', df_full.shape)
threat_type_df = df_full['threat_type'].copy()
# Considering numerical columns
# 34 numerical columns are considered for training
numerical_colmanes = ['duration', 'src_bytes', 'dst_bytes', 'wrong_fragment', 'urgent', 'hot',
                      'num_failed_logins', 'num_compromised', 'root_shell', 'su_attempted', 'num_root',
                      'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'count',
                      'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate',
                      'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
                      'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
                      'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
                      'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate']

numerical_df_full = df_full[numerical_colmanes].copy()
print(numerical_df_full.shape)
#
# # Lets remove the numerical columns with constant value
numerical_df_full = numerical_df_full.loc[:, (numerical_df_full != numerical_df_full.iloc[0]).any()]
#
# # lets scale the values for each column from [0,1]
# # N.B. we dont have any negative values]
final_df_full = numerical_df_full / numerical_df_full.max()
print(final_df_full.shape)

df_normalized = pd.concat([final_df_full, threat_type_df], axis=1)
print('After normalization shape of data set: ', df_normalized.shape)
print(df_normalized[THREAT_TYPE].value_counts())

Before normalization shape of data set :  (148516, 42)
(148516, 34)
(148516, 33)
After normalization shape of data set:  (148516, 34)
0    77053
1    53387
4    14077
3     3880
2      119
Name: threat_type, dtype: int64


In [6]:
def divide_train_test(df, propotion=0.1):
    
    df_train = []
    df_test = []
    for key,val in df[THREAT_TYPE].value_counts().iteritems():
        df_part = df[df['threat_type'] == key]
        df_test.append(df_part[0: int(df_part.shape[0]*propotion)])
        df_train.append(df_part[int(df_part.shape[0]*propotion):df_part.shape[0]])
        
    return df_train,df_test
    

In [7]:
def get_data_for_slices(df_train, number_of_slices, isSmote=False, x_name="x_train", y_name="y_train"):
    
    x_data_dict= dict()
    y_data_dict= dict()    
    
    for i in range(number_of_slices):
        xname= x_name+str(i)
        yname= y_name+str(i)
        df_types = []
        
        for df in df_train:
            df_type = df[int(df.shape[0]*i/number_of_slices):int(df.shape[0]*(i+1)/number_of_slices)]
            df_types.append(df_type)
        
        slice_df = pd.concat(df_types)
        y_info = slice_df.pop('threat_type').values
        x_info = slice_df.values
        y_info = y_info.astype('int')
        
        if isSmote:
            sm = SMOTE(random_state=42)
            x_info, y_info = sm.fit_resample(x_info, y_info)
        
        print('========================================================================================')
        print('\tX part size for slice ' + str(i) + ' is ' + str(x_info.shape))
        print('\tY part size for slice ' + str(i) + ' is ' + str(y_info.shape))
        print('Value types of each class in slice : ' + str(i))
        print(np.unique(y_info,return_counts=True))
        
        x_info = torch.tensor(x_info).float()
        y_info = torch.tensor(y_info).type(torch.LongTensor)
            
        x_data_dict.update({xname : x_info})
        y_data_dict.update({yname : y_info})
        
    return x_data_dict, y_data_dict     

In [8]:
df_train, df_test = divide_train_test(df_normalized,propotion=0.1)
print(df_train)
# print('Value counts in train set : ')
# df_train[THREAT_TYPE].value_counts()
# print('Value counts in test set : ')
# print(df_test[THREAT_TYPE].value_counts())

x_train_dict, y_train_dict = get_data_for_slices(df_train, number_of_slices, isSmote)

df_test = pd.concat(df_test)
y_test = df_test.pop(THREAT_TYPE).values
x_test = df_test.values

print('Test set size is : x => ' + str(x_test.shape) + ' y => ' + str(y_test.shape))
x_test = torch.tensor(x_test).float()
y_test = torch.tensor(y_test.astype('int')).type(torch.LongTensor)

inputs = x_test.shape[1]
outputs = 5
print(inputs,outputs)

[       duration  src_bytes  dst_bytes  wrong_fragment  urgent    hot  \
14478    0.0000     0.0000     0.0000          0.0000  0.0000 0.0000   
14481    0.0000     0.0000     0.0000          0.0000  0.0000 0.0000   
14482    0.0000     0.0000     0.0000          0.0000  0.0000 0.0000   
14483    0.0000     0.0000     0.0000          0.0000  0.0000 0.0000   
14487    0.0000     0.0000     0.0000          0.0000  0.0000 0.0000   
...         ...        ...        ...             ...     ...    ...   
22532    0.0000     0.0000     0.0000          0.0000  0.0000 0.0000   
22534    0.0000     0.0000     0.0000          0.0000  0.0000 0.0000   
22538    0.0000     0.0000     0.0000          0.0000  0.0000 0.0000   
22539    0.0000     0.0000     0.0000          0.0000  0.0000 0.0000   
22541    0.0000     0.0000     0.0000          0.0000  0.0000 0.0000   

       num_failed_logins  num_compromised  root_shell  su_attempted  ...  \
14478             0.0000           0.0000      0.0000     

--------------------------
### <span style="background-color:#F087F9"> Classification Model </span> 

In [9]:
class Net2nn(nn.Module):
    def __init__(self, inputs, outputs):
        super(Net2nn, self).__init__()
        self.fc1=nn.Linear(inputs,200)
        self.fc2=nn.Linear(200,200)
        self.fc3=nn.Linear(200,outputs)
        
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x

In [10]:
class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func

    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        batches = iter(self.dl)
        for b in batches:
            yield (self.func(*b))

In [11]:
def train(model, train_loader, criterion, optimizer):
    model.train()
    train_loss = 0.0
    correct = 0

    for data, target in train_loader:
        output = model(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        prediction = output.argmax(dim=1, keepdim=True)
        correct += prediction.eq(target.view_as(prediction)).sum().item()
        

    return train_loss / len(train_loader), correct/len(train_loader.dataset)

In [12]:
def validation(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            
            test_loss += criterion(output, target).item()
            prediction = output.argmax(dim=1, keepdim=True)
            correct += prediction.eq(target.view_as(prediction)).sum().item()

    test_loss /= len(test_loader)
    correct /= len(test_loader.dataset)

    return (test_loss, correct)

In [25]:
def confusion_mat(model, test_loader):
    y_pred = []
    y_true = []

    # iterate over test data
    for inputs, labels in test_loader:
        output = model(inputs)  # Feed Network

        output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
        y_pred.extend(output)  # Save Prediction

        labels = labels.data.cpu().numpy()
        y_true.extend(labels)  # Save Truth

    cf_matrix = confusion_matrix(y_true, y_pred)
    precisionv = precision_score(y_true,y_pred,average='macro')
    recallv = recall_score(y_true,y_pred,average='macro')
    print('precision value: '+str(precisionv))
    print('recall value: '+ str(recallv))
#     df_cm = pd.DataFrame(cf_matrix, index=[i for i in Counter(y_test)],
#                          columns=[i for i in Counter(y_test)])
#     plt.figure(1)
#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')
#     plt.figure(figsize=(12, 7))

#     sn.heatmap(df_cm, annot=True).set(xlabel='Predicted label', ylabel='True label')
#     plt.savefig('D:\\learning\\PyTorch\\experiment\\cf\\cf_fl_'+str(self.number_of_slices)+'.png')
    print('confusion matrix for normal scenario for slices : ' + str(number_of_slices))
    print(cf_matrix)
    file.write('\ncf matrix for slice :' + str(number_of_slices))
    file.write('\n'+str(cf_matrix))

---------------------------------
### <span style="background-color:#F087F9"> Functions for Federated Averaging </span> 

In [26]:
def create_model_optimizer_criterion_dict(number_of_slices):
    model_dict = dict()
    optimizer_dict= dict()
    criterion_dict = dict()
    
    for i in range(number_of_slices):
        model_name="model"+str(i)
        model_info=Net2nn(inputs, outputs)
        model_dict.update({model_name : model_info })
        
        optimizer_name="optimizer"+str(i)
        optimizer_info = torch.optim.SGD(model_info.parameters(), lr=learning_rate, momentum=momentum)
        optimizer_dict.update({optimizer_name : optimizer_info })
        
        criterion_name = "criterion"+str(i)
        criterion_info = nn.CrossEntropyLoss()
        criterion_dict.update({criterion_name : criterion_info})
        
    return model_dict, optimizer_dict, criterion_dict 

In [27]:
def get_averaged_weights(model_dict, number_of_slices):
   
    fc1_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].fc1.weight.shape)
    fc1_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].fc1.bias.shape)
    
    fc2_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].fc2.weight.shape)
    fc2_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].fc2.bias.shape)
    
    fc3_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].fc3.weight.shape)
    fc3_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].fc3.bias.shape)
    
    with torch.no_grad():
    
    
        for i in range(number_of_slices):
            fc1_mean_weight += model_dict[name_of_models[i]].fc1.weight.data.clone()
            fc1_mean_bias += model_dict[name_of_models[i]].fc1.bias.data.clone()
        
            fc2_mean_weight += model_dict[name_of_models[i]].fc2.weight.data.clone()
            fc2_mean_bias += model_dict[name_of_models[i]].fc2.bias.data.clone()
        
            fc3_mean_weight += model_dict[name_of_models[i]].fc3.weight.data.clone()
            fc3_mean_bias += model_dict[name_of_models[i]].fc3.bias.data.clone()

        
        fc1_mean_weight =fc1_mean_weight/number_of_slices
        fc1_mean_bias = fc1_mean_bias/ number_of_slices
    
        fc2_mean_weight =fc2_mean_weight/number_of_slices
        fc2_mean_bias = fc2_mean_bias/ number_of_slices
    
        fc3_mean_weight =fc3_mean_weight/number_of_slices
        fc3_mean_bias = fc3_mean_bias/ number_of_slices
    
    return fc1_mean_weight, fc1_mean_bias, fc2_mean_weight, fc2_mean_bias, fc3_mean_weight, fc3_mean_bias

In [28]:
def set_averaged_weights_as_main_model_weights_and_update_main_model(main_model,model_dict, number_of_slices):
    fc1_mean_weight, fc1_mean_bias, fc2_mean_weight, fc2_mean_bias, fc3_mean_weight, fc3_mean_bias = get_averaged_weights(model_dict, number_of_slices=number_of_slices)
    with torch.no_grad():
        main_model.fc1.weight.data = fc1_mean_weight.data.clone()
        main_model.fc2.weight.data = fc2_mean_weight.data.clone()
        main_model.fc3.weight.data = fc3_mean_weight.data.clone()

        main_model.fc1.bias.data = fc1_mean_bias.data.clone()
        main_model.fc2.bias.data = fc2_mean_bias.data.clone()
        main_model.fc3.bias.data = fc3_mean_bias.data.clone() 
    return main_model

In [29]:
def compare_local_and_merged_model_performance(number_of_slices):
    accuracy_table=pd.DataFrame(data=np.zeros((number_of_slices,3)), columns=["sample", "local_ind_model", "merged_main_model"])
    for i in range (number_of_slices):
    
        test_ds = TensorDataset(x_test, y_test)
        test_dl = DataLoader(test_ds, batch_size=batch_size * 2)
    
        model=model_dict[name_of_models[i]]
        criterion=criterion_dict[name_of_criterions[i]]
        optimizer=optimizer_dict[name_of_optimizers[i]]
    
        individual_loss, individual_accuracy = validation(model, test_dl, criterion)
        main_loss, main_accuracy =validation(main_model, test_dl, main_criterion )
    
        accuracy_table.loc[i, "sample"]="sample "+str(i)
        accuracy_table.loc[i, "local_ind_model"] = individual_accuracy
        accuracy_table.loc[i, "merged_main_model"] = main_accuracy

    return accuracy_table

In [30]:
def send_main_model_to_nodes_and_update_model_dict(main_model, model_dict, number_of_slices):
    with torch.no_grad():
        for i in range(number_of_slices):
            print('Updating model :' + name_of_models[i] )
            model_dict[name_of_models[i]].fc1.weight.data =main_model.fc1.weight.data.clone()
            model_dict[name_of_models[i]].fc2.weight.data =main_model.fc2.weight.data.clone()
            model_dict[name_of_models[i]].fc3.weight.data =main_model.fc3.weight.data.clone() 
            
            model_dict[name_of_models[i]].fc1.bias.data =main_model.fc1.bias.data.clone()
            model_dict[name_of_models[i]].fc2.bias.data =main_model.fc2.bias.data.clone()
            model_dict[name_of_models[i]].fc3.bias.data =main_model.fc3.bias.data.clone() 
    
    return model_dict

In [31]:
def start_train_end_node_process(number_of_slices):
    for i in range (number_of_slices): 

        print('Federated learning for slice '+ str(i+1))
        train_ds = TensorDataset(x_train_dict[name_of_x_train_sets[i]], y_train_dict[name_of_y_train_sets[i]])
        train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

#         valid_ds = TensorDataset(x_valid_dict[name_of_x_valid_sets[i]], y_valid_dict[name_of_y_valid_sets[i]])
#         valid_dl = DataLoader(valid_ds, batch_size=batch_size * 2)
        
        test_ds = TensorDataset(x_test_dict[name_of_x_test_sets[i]], y_test_dict[name_of_y_test_sets[i]])
        test_dl = DataLoader(test_ds, batch_size= batch_size * 2)
    
        model=model_dict[name_of_models[i]]
        criterion=criterion_dict[name_of_criterions[i]]
        optimizer=optimizer_dict[name_of_optimizers[i]]
    
        print("Subset" ,i)
        for epoch in range(numEpoch):        
            train_loss, train_accuracy = train(model, train_dl, criterion, optimizer)
#             valid_loss, valid_accuracy = validation(model, valid_dl, criterion)
            test_loss, test_accuracy = validation(model, test_dl, criterion)
    
            print("epoch: {:3.0f}".format(epoch+1) + " | train accuracy: {:7.5f}".format(train_accuracy) + " | test accuracy: {:7.5f}".format(test_accuracy))

In [32]:

def start_train_end_node_process_without_print(number_of_slices):
    for i in range (number_of_slices): 

        train_ds = TensorDataset(x_train_dict[name_of_x_train_sets[i]], y_train_dict[name_of_y_train_sets[i]])
        train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

        test_ds = TensorDataset(x_test, y_test)
        test_dl = DataLoader(test_ds, batch_size= batch_size * 2)
    
        model=model_dict[name_of_models[i]]
        criterion=criterion_dict[name_of_criterions[i]]
        optimizer=optimizer_dict[name_of_optimizers[i]]
    
        for epoch in range(numEpoch):        
            train_loss, train_accuracy = train(model, train_dl, criterion, optimizer)
            test_loss, test_accuracy = validation(model, test_dl, criterion)

In [33]:
def start_train_end_node_process_print_some(number_of_slices, print_amount):
    for i in range (number_of_slices): 
        
        print('Federated learning for slice '+ str(i+1))
        train_ds = TensorDataset(x_train_dict[name_of_x_train_sets[i]], 
                                 y_train_dict[name_of_y_train_sets[i]])
        train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

        test_ds = TensorDataset(x_test, y_test)
        test_dl = DataLoader(test_ds, batch_size= batch_size * 2)
    
        model=model_dict[name_of_models[i]]
        criterion=criterion_dict[name_of_criterions[i]]
        optimizer=optimizer_dict[name_of_optimizers[i]]
    
        if i<print_amount:
            print("Subset" ,i)
            
        for epoch in range(numEpoch):
        
            train_loss, train_accuracy = train(model, train_dl, criterion, optimizer)
            test_loss, test_accuracy = validation(model, test_dl, criterion)
            
            if i<print_amount:        
                print("epoch: {:3.0f}".format(epoch+1) + " | train accuracy: {:7.5f}".format(train_accuracy) + " | test accuracy: {:7.5f}".format(test_accuracy))    

In [34]:
# x_train, y_train, x_valid, y_valid,x_test, y_test = map(torch.tensor, (x_train, y_train, x_valid, y_valid, x_test, y_test))


----------------

### <span style="background-color:#F087F9"> Let's examine what would the performance of the centralized model be if the data were not distributed to nodes at all? </span>   

The model used in this example is very simple, different things can be done to improve model performance, such as using more complex models, increasing epoch or hyperparameter tuning. However, the purpose here is to compare the performance of the main model that is formed by combining the parameters of the local models trained on their own data with a centralized model that trained on all training data. In this way, we can gain insight into the capacity of federated learning.


In [35]:
# initial_model = Net2nn()
# initial_optimizer = torch.optim.SGD(initial_model.parameters(), lr=0.01, momentum=0.9)
# initial_criterion = nn.CrossEntropyLoss()

centralized_model = Net2nn(inputs, outputs)
centralized_optimizer = torch.optim.SGD(centralized_model.parameters(), lr=0.01, momentum=0.9)
centralized_criterion = nn.CrossEntropyLoss()

In [36]:
print("------ Centralized Model ------")

train_acc = []
test_acc = []
train_loss = []
test_loss = []

test_ds = TensorDataset(x_test, y_test)
test_dl = DataLoader(test_ds, batch_size=batch_size * 2)

for i in range(number_of_slices):
    centralized_model = Net2nn(inputs,outputs)
    centralized_optimizer = torch.optim.SGD(centralized_model.parameters(), lr=0.01, momentum=0.9)
    centralized_criterion = nn.CrossEntropyLoss()
#     centralized_model = copy.deepcopy(initial_model)
#     centralized_optimizer = copy.deepcopy(initial_optimizer)
#     centralized_criterion = copy.deepcopy(initial_criterion)
    print('Training with slice ' + str(i+1) + ' data' )
    x_name = 'x_train' + str(i)
    y_name = 'y_train' + str(i)
    train_ds = TensorDataset(x_train_dict[x_name], y_train_dict[y_name])
    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

    for epoch in range(numEpoch):
        central_train_loss, central_train_accuracy = train(centralized_model, train_dl, centralized_criterion, centralized_optimizer)
        central_test_loss, central_test_accuracy = validation(centralized_model, test_dl, centralized_criterion)
        
        train_acc.append(central_train_accuracy)
        train_loss.append(central_train_loss)
        test_acc.append(central_test_accuracy)
        test_loss.append(central_test_loss)
        
#         print("epoch: {:3.0f}".format(epoch+1) + " | train accuracy: {:7.4f}".format(central_train_accuracy) + " | test accuracy: {:7.4f}".format(central_test_accuracy))
    print(" | train accuracy: {:7.4f}".format(central_train_accuracy) + " | test accuracy: {:7.4f}".format(central_test_accuracy))
    confusion_mat(centralized_model, test_dl)
    
print("------ Training finished ------")
print('Mean train accuracy: ' + str(sum(train_acc)/len(train_acc)))
print('Mean test accuracy: ' + str(sum(test_acc)/len(test_acc)))


------ Centralized Model ------
Training with slice 1 data
 | train accuracy:  0.9738 | test accuracy:  0.9789


TypeError: can only concatenate str (not "numpy.float64") to str

In [24]:
file.write('\nCentralized Mean train accuracy: ' + str(sum(train_acc)/len(train_acc)))
file.write('\nCentralized Mean test accuracy: ' + str(sum(test_acc)/len(test_acc)))

51


----------------
-----------------
**Data is distributed to nodes**

<!-- ### <span style="background-color:#F087F9"> Datanın nodelara dağıtılması </span>    -->

In [25]:
print(x_train_dict["x_train1"].shape, y_train_dict["y_train1"].shape)
print(x_test.shape, y_test.shape)

torch.Size([33417, 33]) torch.Size([33417])
torch.Size([14849, 33]) torch.Size([14849])


**Main model is created**

In [26]:
main_model = Net2nn(inputs,outputs)
main_optimizer = torch.optim.SGD(main_model.parameters(), lr=learning_rate, momentum=0.9)
main_criterion = nn.CrossEntropyLoss()

**Models,optimizers and loss functions in nodes are defined**

In [27]:
model_dict, optimizer_dict, criterion_dict = create_model_optimizer_criterion_dict(number_of_slices)

**Keys of dicts are being made iterable**

In [28]:
name_of_x_train_sets=list(x_train_dict.keys())
name_of_y_train_sets=list(y_train_dict.keys())

name_of_models=list(model_dict.keys())
name_of_optimizers=list(optimizer_dict.keys())
name_of_criterions=list(criterion_dict.keys())

print(name_of_x_train_sets)
print(name_of_y_train_sets)
print("\n ------------")
print(name_of_models)
print(name_of_optimizers)
print(name_of_criterions)

['x_train0', 'x_train1', 'x_train2', 'x_train3']
['y_train0', 'y_train1', 'y_train2', 'y_train3']

 ------------
['model0', 'model1', 'model2', 'model3']
['optimizer0', 'optimizer1', 'optimizer2', 'optimizer3']
['criterion0', 'criterion1', 'criterion2', 'criterion3']


In [29]:
print(main_model.fc2.weight[0:1,0:5])
print(model_dict["model1"].fc2.weight[0:1,0:5])

tensor([[ 0.0459,  0.0528,  0.0031,  0.0325, -0.0223]],
       grad_fn=<SliceBackward>)
tensor([[ 0.0564,  0.0337,  0.0494, -0.0080, -0.0444]],
       grad_fn=<SliceBackward>)


**Parameters of main model are sent to nodes**  
Since the parameters of the main model and parameters of all local models in the nodes are randomly initialized, all these parameters will be different from each other. For this reason, the main model sends its parameters to the nodes before the training of local models in the nodes begins. You can check the weights below.

In [30]:
model_dict=send_main_model_to_nodes_and_update_model_dict(main_model, model_dict, number_of_slices)

Updating model :model0
Updating model :model1
Updating model :model2
Updating model :model3


In [31]:
print(main_model.fc2.weight[0:1,0:5])
print(model_dict["model1"].fc2.weight[0:1,0:5])

tensor([[ 0.0459,  0.0528,  0.0031,  0.0325, -0.0223]],
       grad_fn=<SliceBackward>)
tensor([[ 0.0459,  0.0528,  0.0031,  0.0325, -0.0223]],
       grad_fn=<SliceBackward>)


**Models in the nodes are trained**

In [32]:
# start_train_end_node_process()
start_train_end_node_process_print_some(number_of_slices, print_amount)

Federated learning for slice 1
Subset 0
epoch:   1 | train accuracy: 0.90938 | test accuracy: 0.94222
epoch:   2 | train accuracy: 0.94712 | test accuracy: 0.95373
epoch:   3 | train accuracy: 0.95302 | test accuracy: 0.96114
epoch:   4 | train accuracy: 0.95715 | test accuracy: 0.96128
epoch:   5 | train accuracy: 0.96047 | test accuracy: 0.97030
epoch:   6 | train accuracy: 0.96247 | test accuracy: 0.96943
epoch:   7 | train accuracy: 0.96490 | test accuracy: 0.97010
epoch:   8 | train accuracy: 0.96553 | test accuracy: 0.97151
epoch:   9 | train accuracy: 0.96693 | test accuracy: 0.97205
epoch:  10 | train accuracy: 0.96741 | test accuracy: 0.96963
epoch:  11 | train accuracy: 0.96843 | test accuracy: 0.97380
epoch:  12 | train accuracy: 0.96882 | test accuracy: 0.97266
epoch:  13 | train accuracy: 0.96867 | test accuracy: 0.97757
epoch:  14 | train accuracy: 0.96986 | test accuracy: 0.97367
epoch:  15 | train accuracy: 0.97097 | test accuracy: 0.97690
epoch:  16 | train accuracy: 0

In [33]:
## As you can see, wieghts of local models are updated after training process
print(main_model.fc2.weight[0,0:5])
print(model_dict["model1"].fc2.weight[0,0:5])

tensor([ 0.0459,  0.0528,  0.0031,  0.0325, -0.0223], grad_fn=<SliceBackward>)
tensor([ 0.0767, -0.0119,  0.0087,  0.0556, -0.0069], grad_fn=<SliceBackward>)


### Let's compare the performance of federated main model, individual local models and centralized model  

**Federated main model vs individual local models before 1st iteration (on distributed test set)**  
Since main model is randomly initialized and no action taken on it yet, its performance is very poor. Please before_acc_table

In [34]:
before_acc_table=compare_local_and_merged_model_performance(number_of_slices=number_of_slices)
before_test_loss, before_test_accuracy = validation(main_model, test_dl, main_criterion)
file.write('\nbefore training main model')
confusion_mat(main_model, test_dl)

main_model= set_averaged_weights_as_main_model_weights_and_update_main_model(main_model,model_dict, number_of_slices) 

after_acc_table=compare_local_and_merged_model_performance(number_of_slices=number_of_slices)
after_test_loss, after_test_accuracy = validation(main_model, test_dl, main_criterion)
file.write('\nafter training main model')
confusion_mat(main_model, test_dl)

confusion matrix for normal scenario for slices : 4
[[4222   41 3024  414    4]
 [1107    2  363 3866    0]
 [   3    0    8    0    0]
 [ 148    0  238    2    0]
 [ 345   42  572  448    0]]
confusion matrix for normal scenario for slices : 4
[[7586   52    0    4   63]
 [  95 5242    0    0    1]
 [   8    0    0    3    0]
 [ 307   20    0   59    2]
 [  76    1    0    0 1330]]


In [35]:
print("Federated main model vs individual local models before FedAvg first iteration")
file.write('\nBefore training federated')
file.write('\n'+str(before_acc_table))
before_acc_table

Federated main model vs individual local models before FedAvg first iteration


,sample,local_ind_model,merged_main_model
0,sample 0,0.9763,0.2851
1,sample 1,0.9577,0.2851
2,sample 2,0.9630,0.2851
3,sample 3,0.9608,0.2851


In [36]:
print("Federated main model vs individual local models after FedAvg first iteration")
file.write('\nAfter training federated')
file.write('\n'+str(after_acc_table))
after_acc_table

Federated main model vs individual local models after FedAvg first iteration


,sample,local_ind_model,merged_main_model
0,sample 0,0.9763,0.9574
1,sample 1,0.9577,0.9574
2,sample 2,0.9630,0.9574
3,sample 3,0.9608,0.9574


**Federated main model vs centralized model before 1st iteration (on all test data)**  
Please be aware that the centralized model gets approximately %98 accuracy on all test data.

In [37]:
print("Before 1st iteration main model accuracy on all test data: {:7.4f}".format(before_test_accuracy))
print("After 1st iteration main model accuracy on all test data: {:7.4f}".format(after_test_accuracy))
print("Centralized model accuracy on all test data: {:7.4f}".format(central_test_accuracy))

Before 1st iteration main model accuracy on all test data:  0.0661
After 1st iteration main model accuracy on all test data:  0.9608
Centralized model accuracy on all test data:  0.9536


This is a single iteration, we can send the weights of the main model back to the nodes and repeat the above steps.
Now let's check how the performance of the main model improves when we repeat the iteration 10 more times.

In [38]:
for i in range(2):
    model_dict=send_main_model_to_nodes_and_update_model_dict(main_model, model_dict, number_of_slices)
    start_train_end_node_process_without_print(number_of_slices)
    main_model= set_averaged_weights_as_main_model_weights_and_update_main_model(main_model,model_dict, number_of_slices) 
    test_loss, test_accuracy = validation(main_model, test_dl, main_criterion)
    print("Iteration", str(i+2), ": main_model accuracy on all test data: {:7.4f}".format(test_accuracy))   

Updating model :model0
Updating model :model1
Updating model :model2
Updating model :model3
Iteration 2 : main_model accuracy on all test data:  0.9733
Updating model :model0
Updating model :model1
Updating model :model2
Updating model :model3
Iteration 3 : main_model accuracy on all test data:  0.9766


In [39]:
confusion_mat(main_model, test_dl)

confusion matrix for normal scenario for slices : 4
[[7629   53    0   16    7]
 [   9 5329    0    0    0]
 [   5    0    6    0    0]
 [ 183    3    1  199    2]
 [  59    0    0   10 1338]]


In [40]:
for i in range(2):
    model_dict=send_main_model_to_nodes_and_update_model_dict(main_model, model_dict, number_of_slices)
    start_train_end_node_process_without_print(number_of_slices)
    main_model= set_averaged_weights_as_main_model_weights_and_update_main_model(main_model,model_dict, number_of_slices) 
    test_loss, test_accuracy = validation(main_model, test_dl, main_criterion)
    print("Iteration", str(i+2), ": main_model accuracy on all test data: {:7.4f}".format(test_accuracy))   

Updating model :model0
Updating model :model1
Updating model :model2
Updating model :model3
Iteration 2 : main_model accuracy on all test data:  0.9754
Updating model :model0
Updating model :model1
Updating model :model2
Updating model :model3
Iteration 3 : main_model accuracy on all test data:  0.9778


In [41]:
file = open(file_name, "a")
confusion_mat(main_model, test_dl)

confusion matrix for normal scenario for slices : 4
[[7635   45    0   21    4]
 [   8 5330    0    0    0]
 [   5    0    6    0    0]
 [ 183    1    0  202    2]
 [  54    0    0    6 1347]]


The accuracy of the centralized model was calculated as approximately 98%. The accuracy of the main model obtained by FedAvg method started from 85% and improved to 94%. In this case, we can say that although the main model obtained by FedAvg method was trained without seeing the data, its performance cannot be underestimated.

In [42]:
file.close()